From previous version

In [ ]:
X = tickerDf[['Open', 'Close_lag1', 'day_of_year', 'day_of_week', 'week', 'month']]
y = tickerDf['day_change_signal']

In [ ]:
split = X.index.max() -dt.timedelta(days=1*365)

X_train = X[X.index < split]
y_train = y[y.index < split]
X_test = X[X.index >= split]
y_test = y[y.index >= split]

In [ ]:
print(f"The shape of the training data is {X_train.shape} features and {y_train.shape}")
print("--------------------------")
print(f"The shape of the testing data is {X_test.shape} features and {y_test.shape}")

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_enc = pd.Series(le.fit_transform(y_train))
y_test_enc = pd.Series(le.transform(y_test))

In [ ]:
y_train_enc.value_counts(normalize=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import r2_score, accuracy_score, classification_report, roc_auc_score, recall_score, precision_score, f1_score, cohen_kappa_score

In [ ]:
perf_dict = dict()
model_dict = {
    'logr_clf':LogisticRegression(random_state=0),
    'svc_clf':SVC(gamma='auto'), 
}

In [ ]:
def logPerf(y_pred, y_proba, y_test):

    # Log performance metrics
    perf_dict = {
        'accuracy': accuracy_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_proba, multi_class='ovr'),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1': f1_score(y_test, y_pred, average='weighted'),
        'kappa': cohen_kappa_score(y_test, y_pred)
    }

    ret_dict = {
        'y_pred': y_pred,
        'y_proba':y_proba,
        'perf_results': perf_dict
    }

    return ret_dict

In [ ]:
for clf in model_dict:
    print(clf)
    # Define classifer
    clf = model_dict[i].fit(X_train, y_train_enc)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)
    perf_dict[clf] = logPerf(y_pred, y_proba, X_test)

In [ ]:
model_name = 'logr_clf'

# Define classifer
clf = LogisticRegression(random_state=0).fit(X_train, y_train_enc)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

# Log performance metrics
perf_dict[model_name] = {
    'accuracy': accuracy_score(y_test_enc, y_pred),
    'auc': roc_auc_score(y_test_enc, y_proba, multi_class='ovr'),
    'precision': precision_score(y_test_enc, y_pred, average='weighted'),
    'recall': recall_score(y_test_enc, y_pred, average='weighted'),
    'f1': f1_score(y_test_enc, y_pred, average='weighted'),
    'kappa': cohen_kappa_score(y_test_enc, y_pred)
}

In [ ]:
pd.DataFrame(perf_dict).T